In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("综合设计").master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled",True) # 设置dataframe的显示格式

In [ ]:
df = spark.read.option("inferSchema","true").option("header","true").csv("/data/data.csv") # 读取csv文件

In [ ]:
df

In [ ]:
df = df.drop("_c0")

In [ ]:
df

### 预处理

In [ ]:
from pyspark.sql.functions import split,col,regexp_replace

In [ ]:
df = df.withColumn("xiaoqu",split(col("address")," ")[0])

In [ ]:
df

In [ ]:
regex = "^\D\D+"

In [ ]:
df = df.withColumn("BuildingTime",regexp_replace(col("BuildingTime"),regex,''))

In [ ]:
df

In [2]:
df.write.format("csv").option("sep",',').option("header","true").save("/data/dataprocessed")

NameError: name 'df' is not defined

In [3]:
df = spark.read.option("inferSchema","true").option("header","true").csv("/data/dataprocessed") # 读取csv文件

In [ ]:
df.select("BuildingArea","SubwayDistance","price_total","price_square","price_dy").describe()

In [ ]:
df.select("BuildingArea","SubwayDistance","price_total","price_square","price_dy").summary()

### 房价最大值、最小值、平均值、中位数

In [4]:
import plotly.express as px

In [ ]:
from pyspark.sql.functions import min,max,mean,expr,desc

In [ ]:
import pyecharts.options as opts
from pyecharts.charts import Bar, Line

In [ ]:
df.select("BuildingArea","SubwayDistance","price_total","price_square","price_dy").toPandas().quantile()

In [ ]:
result = df.select(min("price_total").alias("房屋总价最小值(万)"),max("price_total").alias("房屋总价最大值(万)")
          ,mean("price_total").alias("房屋总价平均值(万)"),expr("percentile(price_total, array(0.5))[0]").alias("房屋总价中位数(万)")
           ,min("price_square").alias("房屋单位面积价格最小值(元/m2)"),max("price_square").alias("房屋单位面积价格最大值(元/m2)")
          ,mean("price_square").alias("房屋单位面积价格平均值(元/m2)"),
          expr("percentile(price_square, array(0.5))[0]").alias("单位面积房屋价格中位数(元/m2)"))

In [ ]:
result

In [ ]:
result

result_dict = result.toPandas().to_dict()

In [ ]:
keys = [key for key in result_dict.keys()]

In [ ]:
values = [value[0] for value in result_dict.values()]

In [ ]:
x = ["最小值","最大值","平均值","中位数"]

In [ ]:
values

In [ ]:
bar = Bar()
bar.add_xaxis(x)
bar.add_yaxis("二手房房屋总价(万)",values[:4])
bar.add_yaxis("二手房房屋单位面积价格(元/m2)",values[4:])
bar.render_notebook()

In [ ]:
df.groupBy("xiaoqu").agg(min("price_total").alias("房屋总价最小值(万)"),max("price_total").alias("房屋总价最大值(万)")
          ,min("price_square").alias("房屋单位面积价格最小值(元/m2)"),max("price_square").alias("房屋单位面积价格最大值(元/m2)"),
          expr("percentile(price_total, array(0.5))[0]").alias("房屋总价中位数(万)"),
          expr("percentile(price_square, array(0.5))[0]").alias("单位面积房屋价格中位数(元/m2)"))

In [ ]:
df.groupBy("xiaoqu").agg(expr("percentile(price_total, array(0.5))[0]").alias("每个小区房屋总价中位数(万)"),
                        expr("percentile(price_square, array(0.5))[0]").alias("每个小区单位面积房屋价格中位数(元/m2)"))

In [ ]:
df.select(expr("percentile(price_total, array(0.5))[0]").alias("合肥市房屋总价中位数(万)"),
         expr("percentile(price_square, array(0.5))[0]").alias("合肥市房屋单位面积价格中位数(万/m2)"))

In [ ]:
df.select("address","price_total","price_square","labels","xiaoqu").orderBy(desc("price_square")).limit(20) # 单位面积最高价格

In [ ]:
df.select("address","price_total","price_square","labels").orderBy(desc("price_total")).limit(20) # 房屋总价的最高价格

In [ ]:
df.orderBy("price_square").limit(20) # 单位面积最低价格

In [ ]:
df.orderBy("price_total").limit(20) # 单位面积最低价格

In [ ]:
pd_df = df.groupBy("xiaoqu").mean("price_square","price_total").orderBy(desc("avg(price_square)")).limit(20).toPandas()

In [ ]:
df.groupBy("xiaoqu").mean("price_square","price_total").orderBy(desc("avg(price_square)")).limit(20)

In [ ]:
pd_df

In [ ]:
pd_df.to_dict('list')

In [ ]:
x = pd_df_dict['xiaoqu']

In [ ]:
y1 = pd_df_dict['avg(price_square)']
y2 = pd_df_dict['avg(price_total)']

In [ ]:
bar = (Bar().add_xaxis(xaxis_data=x).add_yaxis(series_name="房屋总价",y_axis= y2,).extend_axis(yaxis=opts.AxisOpts(
        name="二手房房屋总价平均值",type_="value",axislabel_opts=opts.LabelOpts(formatter="{value} 元/m2"),))
    .set_global_opts(tooltip_opts=opts.TooltipOpts(is_show=True, trigger="axis", axis_pointer_type="cross"),
    xaxis_opts=opts.AxisOpts(type_="category",axispointer_opts=opts.AxisPointerOpts(is_show=True, type_="shadow"),
            axislabel_opts={"rotate":45}),yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} 万")),))
line = (Line().add_xaxis(xaxis_data=x).add_yaxis(series_name="房屋单位面积价格",yaxis_index=1,y_axis=y1
                                                 ,label_opts=opts.LabelOpts(is_show=False),))
bar.overlap(line).render_notebook()

In [ ]:
from pyspark.sql.window import Window
windowXiaoqu = Window.partitionBy("xiaoqu").orderBy(desc("price_square"))

In [ ]:
from pyspark.sql.functions import dense_rank
df.withColumn("rk",dense_rank().over(windowXiaoqu)).where(col("rk")<=3)

In [ ]:
import plotly.express as px

In [ ]:
pd_df = df.toPandas()

In [ ]:
fig = px.scatter(pd_df,x='xiaoqu',y='price_square',color='xiaoqu',size = 'price_total')
fig.show()

In [ ]:
fig = px.box(pd_df,x='huxing',y='price_square',color='huxing')
fig.show()

In [12]:
df

title,address,huxing,BuildingArea,Position,Direction,BuildingTime,SubwayDistance,price_total,price_square,price_dy,labels,xiaoqu
富世广场 红星路42电梯房 146...,富世广场 黄山大厦-宿州路与红星路...,1室2厅,63,中层（共29层）,北向,2012年建,2,146,23174,null,"满二,优质教育",富世广场
"相府花园 步行街相府花园,三室两厅...",相府花园 市府广场-淮河路118号,3室1厅,91,高层（共7层）,南向,2000年建,2,195,21381,6,满五,相府花园
海伦国际 4.5米挑高精装修2室送...,滨湖海伦堡 新省府-滨湖海伦堡锦绣...,2室1厅,46,高层（共30层）,南向,null,1,45,9782,null,"满两年,好房推荐,精装",滨湖海伦堡
滨湖区 招商城市主场 央企开发商 ...,招商城市主场(公寓) 新省府-招商...,2室1厅,55,高层（共26层）,南向,2022年建,null,56,10181,null,null,招商城市主场(公寓)
美菱新村 2室1厅64.65m2满五,美菱新村 双岗-双岗濉溪路桥旁,2室1厅,64,低层（共7层）,南向,2000年建,null,95,14694,null,"满五,优质教育",美菱新村
文昌雅居 3室2厅1厨1卫 114...,文昌雅居 周谷堆-淝河路与周谷堆路交口,3室2厅,114,低层（共28层）,南向,2011年建,null,150,13137,null,"满二,优质教育",文昌雅居
文一云湖轩 董铺湖景房 南北向精装...,文一云湖轩 大铺头-科学岛路与石牛路交口,3室1厅,111,底层（共11层）,南北向,null,null,260,23423,null,null,文一云湖轩
钢铁新村 2室1厅43.45m2 ...,钢铁二村 合瓦路-临泉路与合瓦路交汇处,2室1厅,43,高层（共6层）,南向,1992年建,3,60,13808,null,满五,钢铁二村
墨荷名邸 3室2厅98m2精装修,墨荷名邸 森林公园-北二环与四里河...,3室2厅,98,高层（共34层）,南北向,2008年建,null,169,17244,null,null,墨荷名邸
恒大中央广场 3室2厅93m2精装修,恒大中央广场 汽车站-明光路和胜利...,3室2厅,93,中层（共27层）,南向,2000年建,1,240,25806,null,满五,恒大中央广场


In [14]:
pd_df = df.toPandas()
fig = px.pie(pd_df,values='BuildingArea',names='xiaoqu')
fig.update_traces(textposition='inside', 
                  textinfo='percent+label'
                 )

AttributeError: 'DataFrame' object has no attribute 'mean'

In [33]:
pd_new_df

BuildingTime,avg(price_square)
2016年建,20899.720930232557
2003年建,18254.25
1996年建,10625.6
2014年建,20451.394366197183
1986年建,32177.666666666668
1999年建,21615.571428571428
2018年建,20286.983606557376
1979年建,24050.0
2017年建,21593.925
2013年建,21660.985074626864


In [39]:
fig = px.scatter(pd_df.sort_values(by="BuildingTime",ascending=False),x='BuildingTime',y='price_square',
                     color='xiaoqu',size='price_total'
                    )
fig.show()

In [40]:
fig = px.violin(pd_df.sort_values(by="BuildingTime",ascending=False),x='BuildingTime',y='price_square',
                     color='BuildingTime',box=True
                    )
fig.show()

In [43]:
fig = px.sunburst(df.groupBy("Direction","huxing").count().toPandas(),path=['Direction','huxing'],values='count',title="不同朝向各户型数量")
fig.show()